# Прогнозирование температуры звезды

## Описание проекта
**Задача от обсерватории «Небо на ладони»**: разработать нейросеть для определения температуры на поверхности обнаруженных звёзд.

### Цель проекта
Создать модель машинного обучения, которая будет предсказывать абсолютную температуру звезды на основе её характеристик.

---

## Традиционные методы расчёта температуры
Обычно учёные используют:
1. **Закон смещения Вина**
2. **Закон Стефана-Больцмана** 
3. **Спектральный анализ**

Каждый метод имеет преимущества и недостатки. ML-подход должен стать более точной и удобной альтернативой.

---

## Данные для обучения
В базе обсерватории есть характеристики **240 изученных звёзд**:

### Признаки звезды:
| Признак | Описание |
|---------|-----------|
| **Относительная светимость** | L/Lo - светимость звезды относительно Солнца |
| **Относительный радиус** | R/Ro - радиус звезды относительно радиуса Солнца |
| **Абсолютная звёздная величина** | Mv - физическая величина, характеризующая блеск звезды |
| **Звёздный цвет** | white, red, blue, yellow, yellow-orange и др. |
| **Тип звезды** | Категориальный признак |

### Типы звезд:
| Тип звезды | Числовой код |
|------------|--------------|
| Коричневый карлик | 0 |
| Красный карлик | 1 |
| Белый карлик | 2 |
| Звёзды главной последовательности | 3 |
| Сверхгигант | 4 |
| Гипергигант | 5 |

### Целевая переменная:
- **Абсолютная температура T(K)** - температура на поверхности звезды в Кельвинах

# Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from IPython.display import HTML, display
import seaborn as sns
import re
import warnings
import logging
import optuna

# === Библиотеки ML ===
from sklearn.model_selection import train_test_split

# === Функции проекта ===
from func import (
    EarlyStoppingCallback,
    check_data,
    drop_duplicated,
    plot_combined,
    calc_target_correlations,
)

# === Настройки ===
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    module="sklearn.feature_selection._univariate_selection",
)
logging.getLogger("sklearn").setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.WARNING)
pd.set_option("display.max_columns", None)

xgb_params = {"verbosity": 0}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore", category=UserWarning, module="shap")

# Константы

In [ ]:
# зафиксируем константы
RANDOM_STATE = 20
TEST_SIZE = 0.25
N_JOBS = -1
N_ITER = 10 # число итераций для перебора и поиска лучших параметров
N_CROSS_VALL = 3

# Загрузка данных

In [ ]:
df = pd.read_csv("../data/6_class.csv")

# Анализ и предобработка данных

In [ ]:
df = check_data(df)

In [ ]:
# опять непонятный unnamed:_0 который дублирует индекс
df = df.drop(columns=['unnamed:_0'])

Пропусков нет, но есть цвет звезды - проверим

In [ ]:
df['star_color'].unique()

Так и оказалось, заполняли непонятно как... приведем неявные дубликаты к общему виду и немного наведем красоты

In [ ]:
# PS: как именно обозначал автор цвета звезд, не совсем понятно.
# Возьмем первое слово = преобладающий цвет
color_mapping = {
    'blue': 'blue',
    'blue white': 'blue', 
    'blue-white': 'blue',
    
    'white': 'white',
    'whitish': 'white',
    'white-yellow': 'white',
    
    'yellow-white': 'yellow',
    'yellowish': 'yellow',
    'yellowish white': 'yellow',
    'pale yellow orange': 'yellow',
    
    'orange': 'orange',
    'orange-red': 'red',
    
    'red': 'red'
}

df['star_color'] = df['star_color'].str.strip().str.lower().replace(color_mapping)
df['star_color'].unique()

In [ ]:
df.sample(5)

Да, цвет звезды не совсем бьется с ее "цветовым типом", но в целом все логично.

In [ ]:
# и теперь проверим явные дубликаты после корректировок
df = drop_duplicated(df)

Посмотрим на распределение

In [ ]:
plot_combined(df, col=None, target=None, col_type=None, legend_loc='best')

Распределение по цвету звёзд получилось идеальным, значит теории о том, что первое слово важнее второго, подтверждается. :)  
А вот остальные признаки требуют дополнительного изучения и, наверное, обработки.

In [ ]:
# очень много данных temperature_(_k) меньше 5000, посмотрим на них отдельно
df_low_temp = df[df['temperature_(_k)'] < 2000]
df_low_temp

Нулевых нет и на том спасибо.  
К томуже справочный материал говорит, что такая температура вполне возможна.

In [ ]:
# и выбросы по высокой температуре
df_high_temp = df[df['temperature_(_k)'] > 35000]
df_high_temp

Вопросики по светимости к типу 4 да и в целом все еще непонятно определение цвета звезд.  
Разные типы, разные размеры, разные светимости....  
Ничего непонятно, но очень интересно :)

Теперь свечение luminosity(_l/_lo)

In [ ]:
df_low_lum = df[df['luminosity(_l/_lo)'] < 100]
df_low_lum

In [ ]:
df_low_lum = df[df['luminosity(_l/_lo)'] > 500000]
df_low_lum

Вот о чем и писал выше - светимость огромная, а температура может быть не очень высокой.

Так как у нас идет сравнение с Солнцем, что оставим как есть.  

Радиус radius(_r/_ro) скорее всего покажет схожую картину, но посмотрим.

In [ ]:
df_low_rad = df[df['radius(_r/_ro)'] < 100]
df_low_rad

In [ ]:
df_low_rad = df[df['radius(_r/_ro)'] > 200]
df_low_rad

In [ ]:
calc_target_correlations(df, target_col="temperature_(_k)")

Мультиколлинеарности не наблюдается.  
VIF в пределах допустимого.

## Выводы по анализу данных

**Основные проблемы в данных о звездах**  

*Несоответствие температуры и цвета*  
-- Холодные звезды указаны с цветами, характерными для горячих звезд  
-- Нарушена физическая связь между температурой и спектральным классом  

*Нарушение астрофизических законов*  
-- Не соблюдается закон Стефана-Больцмана (связь светимости с радиусом и температурой)  
-- Параметры светимости и радиуса противоречат друг другу  

*Статистические аномалии*  
-- Слишком много редких типов звезд (гипергигантов)  
-- Нарушены реальные пропорции встречаемости разных классов звезд  

*Проблемы с классификацией*  
-- Цвета звезд не соответствуют их физическим параметрам  
-- Отсутствует системный подход к классификации по спектральным классам  
-- Произвольное сочетание характеристик без физического обоснования  

*Методологические проблемы*  
-- Данные выглядят сгенерированными, а не наблюдаемыми  
-- Отсутствует внутренняя согласованность параметров  
-- Нарушены известные астрофизические зависимости  

В целом: Данные требуют серьезной доработки для соответствия реальным астрономическим закономерностям.  
Но ввиду того, что это учебный проект, будем работать с тем, что есть.  

# Построение базовой нейронной сети

## Разделение данных на признаки и целевой признак

In [ ]:
# разделим на выборки их оставшихся данных
target = 'temperature_(_k)'
X_train, X_val, y_train, y_val = train_test_split(
    df.drop([target], axis=1),
    df[target],
    test_size=TEST_SIZE,
    stratify=df[target],
    random_state=RANDOM_STATE
)

# Улучшение нейронной сети

# Выводы